In [1]:
from bs4 import BeautifulSoup as bs
import urllib.request
import re
import pickle
import json

In [2]:
all_albums = []

for year in range(2002,2018) :
    
    url = "https://www.billboard.com/charts/year-end/"+str(year)+"/top-billboard-200-albums"
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib.request.urlopen(req)
    res = bs(html.read(),"html5lib")
    album_titles = [re.sub("\n", '', c.text) for c in res.findAll(attrs={'class': "ye-chart-item__title"})]
    album_artists = [re.sub("\n", '', c.text) for c in res.findAll(attrs={'class': "ye-chart-item__artist"})]

    all_albums.extend([{'album': x[0], 'artist' : x[1], 'year' : year, 'rank' : r + 1} for r, x in enumerate(zip(album_titles, album_artists))])
    
all_albums[:10]

[{'album': 'The Eminem Show', 'artist': 'Eminem', 'year': 2002, 'rank': 1},
 {'album': 'Weathered', 'artist': 'Creed', 'year': 2002, 'rank': 2},
 {'album': 'Nellyville', 'artist': 'Nelly', 'year': 2002, 'rank': 3},
 {'album': 'M!ssundaztood', 'artist': 'P!nk', 'year': 2002, 'rank': 4},
 {'album': '[Hybrid Theory]',
  'artist': 'Linkin Park',
  'year': 2002,
  'rank': 5},
 {'album': 'O Brother, Where Art Thou?',
  'artist': 'Soundtrack',
  'year': 2002,
  'rank': 6},
 {'album': 'Silver Side Up', 'artist': 'Nickelback', 'year': 2002, 'rank': 7},
 {'album': 'Britney', 'artist': 'Britney Spears', 'year': 2002, 'rank': 8},
 {'album': 'Now 8', 'artist': 'Various Artists', 'year': 2002, 'rank': 9},
 {'album': 'Word Of Mouf', 'artist': 'Ludacris', 'year': 2002, 'rank': 10}]

# save songs

In [3]:
with open('all_albums.json', 'w') as fp:
    json.dump(all_albums, fp)